# Setting up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ta
# from fastai import *
# from fastai.tabular import *
from sklearn.ensemble import RandomForestRegressor
from rolling import RollingWindowSplit
from sklearn.metrics import r2_score as r2d2
from joblib import dump, load
from datetime import datetime, timedelta

%matplotlib inline
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# %%time
# path = 'D://Coding//XTX Forecasting Challenge//data-training.csv'
# df = pd.read_csv(path)

In [43]:
path = 'D://Coding//XTX Forecasting Challenge//data-training.file'
df = pd.read_feather(path, use_threads=8)
df = df.astype('float32')
df.fillna(0, inplace=True)

In [3]:
bidSizeList = ['bidSize' + str(i) for i in range(0,15)]
askSizeList = ['askSize' + str(i) for i in range(0,15)]
bidRateList = ['bidRate' + str(i) for i in range(0,15)]
askRateList = ['askRate' + str(i) for i in range(0,15)]

# Exploratory Data Analysis

In [ ]:
# # Figuring out what [y] is
# # y(t) is midRate(t+87) - midRate(t), clipped to (-5.5)
# df['expectedY'] = df.midRate.diff(87).shift(-87).clip(-5,5)

# Feature engineering

### Basics

#### Cross-sectional features

In [5]:
# different from submission
def compute_cross_sectional(df):
    df = pd.DataFrame([base_row])
    df.columns = [*askRateList, *askSizeList, *bidRateList, *bidSizeList]

    # Cross-sectional features
    df['spread'] = df.askRate0 - df.bidRate0
    df['midRate'] = (df.askRate0 + df.bidRate0) / 2
    df['bidAskVol'] = df.askSize0 + df.bidSize0
    df['totalBidVol1'] = df.bidSize0 + df.bidSize1
    df['totalAskVol1'] = df.askSize0 + df.askSize1
    for i in range(2,15):
        df['totalBidVol' + str(i)] = df['totalBidVol' + str(i-1)] + df['bidSize' + str(i)]
        df['totalAskVol' + str(i)] = df['totalAskVol' + str(i-1)] + df['askSize' + str(i)]
    for i in range(1,15):
        df['bidAskRatio' + str(i)] = df['totalBidVol' + str(i)] / df['totalAskVol' + str(i)]
    df['totalAvailVol'] = df.totalBidVol14 + df.totalAskVol14
    df['vwaBid'] = np.einsum('ij,ji->i', df[bidRateList], df[bidSizeList].T) / df[bidSizeList].sum(axis=1)
    df['vwaAsk'] = np.einsum('ij,ji->i', df[askRateList], df[askSizeList].T) / df[askSizeList].sum(axis=1)
    df['vwaBidDMid'] = df.midRate - df.vwaBid
    df['vwaAskDMid'] = df.vwaAsk - df.midRate
    df['diff_vwaBidAskDMid'] = df.vwaAskDMid - df.vwaBidDMid
    return df
# df = compute_cross_sectional(df)

# TA

#### Time series features

In [6]:
def add_time_features(df):
    b1, a1 = (df.bidRate0 < df.bidRate0.shift(1)), (df.askRate0 < df.askRate0.shift(1))
    b2, a2 = (df.bidRate0 == df.bidRate0.shift(1)), (df.askRate0 == df.askRate0.shift(1))
    valsB, valsA = [0, (df.bidSize0 - df.bidSize0.shift(1))], [0, (df.askSize0 - df.askSize0.shift(1))]
    defaultB, defaultA = df.bidSize0, df.askSize0
    df.fillna(0, inplace=True)
    df['deltaVBid'] = np.select([b1,b2], valsB, default=defaultB)
    df['deltaVAsk'] = np.select([a1,a2], valsA, default=defaultA)
    df['VOI'] = df.deltaVBid - df.deltaVAsk
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    return df
# df = add_time_features(df)

In [ ]:
# Requires a window of up to a 1000 past items

#### Manual time features — can consider adding more to the lags list

In [7]:
def add_manual_time_features(df):
    lags = [*np.arange(1,10), *np.arange(10,100,10), *np.arange(100,1000,100)]
    def addTimeFeatures(i):
        df['daskRate' + str(i)] = df.askRate0.diff(i)
        df['dbidRate' + str(i)] = df.bidRate0.diff(i)
    for i in lags:
        addTimeFeatures(i)
    df.fillna(0, inplace=True)
    return df
# df = add_manual_time_features(df)

In [ ]:
df.to_feather('intermediate.file')

In [37]:
df = pd.read_feather('intermediate.file')

#### Tick chart version with ffill

In [38]:
# midrate version
df['time'] = pd.date_range(start='1/1/1970', periods=2999999, freq='T')
df.set_index('time', inplace=True)
df_mid = df.midRate.resample('15Min').ohlc()
df_mid['vol'] = df.bidAskVol.resample('15Min').mean()

In [ ]:
# takes 5 min
df_mid_ta = ta.add_all_ta_features(df_mid, "open", "high", "low", "close", "vol", fillna=True)

In [14]:
# dump(df_mid_ta, 'df_mid_ta.joblib')
df_mid_ta = load('df_mid_ta.joblib')

In [ ]:
# takes 30s
new_df = df.join(df_mid_ta).ffill()
new_df = new_df.astype('float32')

In [35]:
# dump(new_df, 'new_df.joblib')
new_df = load('new_df.joblib')

In [36]:
new_df

,askRate0,askRate1,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askRate8,askRate9,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14,y,spread,midRate,bidAskVol,totalBidVol1,totalAskVol1,totalBidVol2,totalAskVol2,totalBidVol3,totalAskVol3,totalBidVol4,totalAskVol4,totalBidVol5,totalAskVol5,totalBidVol6,totalAskVol6,totalBidVol7,totalAskVol7,totalBidVol8,totalAskVol8,totalBidVol9,totalAskVol9,totalBidVol10,totalAskVol10,totalBidVol11,totalAskVol11,totalBidVol12,totalAskVol12,totalBidVol13,totalAskVol13,totalBidVol14,totalAskVol14,bidAskRatio1,bidAskRatio2,bidAskRatio3,bidAskRatio4,bidAskRatio5,bidAskRatio6,bidAskRatio7,bidAskRatio8,bidAskRatio9,bidAskRatio10,bidAskRatio11,bidAskRatio12,bidAskRatio13,bidAskRatio14,totalAvailVol,vwaBid,vwaAsk,vwaBidDMid,vwaAskDMid,diff_vwaBidAskDMid,deltaVBid,deltaVAsk,VOI,OIR,daskRate1,dbidRate1,daskRate2,dbidRate2,daskRate3,dbidRate3,daskRate4,dbidRate4,daskRate5,dbidRate5,daskRate6,dbidRate6,daskRate7,dbidRate7,daskRate8,dbidRate8,daskRate9,dbidRate9,daskRate10,dbidRate10,daskRate20,dbidRate20,daskRate30,dbidRate30,daskRate40,dbidRate40,daskRate50,dbidRate50,daskRate60,dbidRate60,daskRate70,dbidRate70,daskRate80,dbidRate80,daskRate90,dbidRate90,daskRate100,dbidRate100,daskRate200,dbidRate200,daskRate300,dbidRate300,daskRate400,dbidRate400,daskRate500,dbidRate500,daskRate600,dbidRate600,daskRate700,dbidRate700,daskRate800,dbidRate800,daskRate900,dbidRate900,open,high,low,close,vol,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_vpt,volume_nvi,volatility_atr,volatility_bbh,volatility_bbl,volatility_bbm,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kchi,volatility_kcli,volatility_dch,volatility_dcl,volatility_dchi,volatility_dcli,trend_macd,trend_macd_signal,trend_macd_diff,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,momentum_rsi,momentum_mfi,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00,1619.5,1620.0,1621.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,35.0,10.0,1.0,10.0,13.0,-0.5,4.5,1617.25,8.0,17.0,11.0,18.0,35.0,28.0,35.0,48.0,35.0,51.0,35.0,71.0,35.0,98.0,35.0,109.0,35.0,123.0,35.0,158.0,35.0,168.0,35.0,169.0,35.0,179.0,35.0,192.0,35.0,1.545455,0.514286,0.800000,1.371429,1.457143,2.028571,2.800000,3.114286,3.514286,4.514286,4.800000,4.828571,5.114286,5.485714,227.0,1606.132812,1620.671387,11.117188,3.421387,-7.695801,7.0,1.0,6.0,0.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1617.25,1617.25,1617.25,1617.25,8.000000,-0.007724,0.0,0.000000,0.0,0.000000,-0.119901,1000.000000,0.248556,1617.250000,1617.25000,1617.250000

# Feature Selection

In [ ]:
# figure out a way to validate the blend

In [ ]:
new_df.drop([*askRateList, *askSizeList, *bidRateList, *bidSizeList], axis=1, inplace=True)

In [ ]:
# takes 40s
X = new_df.drop('y', axis=1).values
y = new_df.y.values

# standardise
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# pca takes 1 min
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
dump(scaler, 'scaler_drop.joblib')
dump(pca, 'pca_drop.joblib')
# X_pca = load('X_pca.joblib')
# y = load('y.joblib')

In [ ]:
# print(pca.explained_variance_ratio_)

# Cross-validation

# Lasso

In [ ]:
rlcv = RollingWindowSplit(n_splits=5, compatible=True)

In [ ]:
# takes at least 1 min on pca variables
from sklearn.linear_model import LassoLarsCV
lasso = LassoLarsCV(cv=rlcv, n_jobs=-1).fit(X_pca, y)

In [ ]:
# actually the lasso above has seen the entire dataset....

In [ ]:
dump(lasso, 'lassocv_drop.joblib')
# lasso = load('lassocv.joblib')

In [ ]:
lasso

In [ ]:
def rlcvscore(model):
    cvtrain, cvvalid, cvvalidsig = [], [], []
    for inc, (train_index, valid_index) in enumerate(rlcv.split(X_pca), 1):
        x_train, x_valid = X_pca[train_index], X_pca[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        cvtrain.append(model.score(x_train, y_train))
        cvvalid.append(model.score(x_valid, y_valid))
        sigmoid = (1/(1+np.exp(-0.22*model.predict(x_valid)))-0.5)*20  
        cvvalidsig.append(r2d2(y_valid, sigmoid))
    print(f'{np.array(cvtrain).round(4)}')
    print(f'{np.array(cvvalid).round(4)}')
    print(f'{np.array(cvvalidsig).round(4)}')
    print(f'{np.mean(cvtrain):.4f}, {np.mean(cvvalid):.4f}, {np.mean(cvvalidsig):.4f}')

In [ ]:
rlcvscore(lasso) # dropped

In [ ]:
rlcvscore(lasso) # undropped

In [ ]:
# dump(lasso, f'lasso_rlcv_114ft_0.0175_0.0187.joblib')
# lasso = load('lasso_rlcv_114ft_0.0175_0.0187.joblib') 

# RF

In [ ]:
rf_model = RandomForestRegressor(n_estimators=10, max_depth=6, min_samples_split=1000, min_samples_leaf=1000,
                                 max_features='auto', n_jobs=-1, random_state=41)

In [ ]:
rf_model.fit(x_train, y_train);

In [ ]:
rlcvscore(rf_model) # realistic cv

In [ ]:
a = df.drop('y', axis=1).columns[indices]

In [ ]:
# create X with important variables only
X = df.drop('y', axis=1)[a[:20]].values
y = df.y.values

In [ ]:
rf_model = RandomForestRegressor(n_estimators=10, max_depth=2, min_samples_split=2, min_samples_leaf=5000,
                                 max_features='auto', n_jobs=-1, random_state=41)
rf_model.fit(x_train, y_train);

In [ ]:
rlcvscore(rf_model) #n_est 10, depth 2, samples_split 2, samples_leaf 1000, 30 most importantvariables

In [ ]:
importances = rf_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Plot the feature importances of the forest
plt.figure(figsize=(15,8))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
# save model
dump(rf, 'model.joblib')

In [ ]:
# load model
rf2 = load('model.joblib')

# Fast.ai

In [ ]:
dep_var = 'y'
procs = [FillMissing, Normalize]

In [ ]:
path = f'D:\Coding\XTX Forecasting Challenge'
data = TabularDataBunch.from_df(path = path, df = df[:int(5e5)], dep_var = 'y', procs=procs,
                                 valid_idx = list(range(int(4e5),int(5e5))))

In [ ]:
data.show_batch(rows=10)

In [ ]:
# data = (TabularList.from_df(df[:int(5e5)], cont_names=df.columns, procs=procs)
#                            .split_by_idx(list(range(int(0.8*5e5),int(5e5))))
#                            .label_from_df(cols=dep_var, label_cls=FloatList)
#                            .databunch())

In [ ]:
learn = tabular_learner(data, layers=[500,200], metrics=r2_score, ps=[0.001,0.01], emb_drop=0.04)

In [ ]:
learn.model

In [ ]:
learn.lr_find(end_lr=1e1)

In [ ]:
learn.recorder.plot()

In [ ]:
# model above has already diverged, we will restart.

In [ ]:
learn.fit_one_cycle(3, 1e-4, wd=0.1)

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.save('new_fastai')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.predict(df.iloc[int(8.1e5)])

In [ ]:
df.y.iloc[int(8.1e5)]

In [ ]:
preds = learn.get_preds()

# Submission testing

In [8]:
def get_next_data_as_numpy_array(iteration):
    return df.iloc[iteration][:60].values

In [9]:
def append_to_df(massive_df, row):
    try: row.index = [massive_df.index[-1] + timedelta(minutes=1)]
    except: row.index = [datetime(1970,1,1)]
    return massive_df.append(row)

In [10]:
def add_resample_features(massive_df, resampled_df):
    leftovers = len(massive_df) % 15
    a = pd.DataFrame()
    def pad_history():
        full_resampled = resampled_df.append(df_mid)
        a = pd.DataFrame([full_resampled.iloc[0] for j in range(30+1-len(full_resampled))])
        a = a.append(full_resampled)
        a.index = pd.date_range(start=df_mid.index[-1], periods=len(a), freq='-15Min').sort_values()
        df_mid_ta = ta.add_all_ta_features(a, "open", "high", "low", "close", "vol", fillna=True)
        return df_mid_ta
    if leftovers == 0:
        df_mid = massive_df.tail(15).midRate.resample('15Min').ohlc()
        df_mid['vol'] = massive_df.tail(15).bidAskVol.resample('15Min').mean()
        df_mid_ta = pad_history()
        resampled_df = resampled_df.append(df_mid_ta)
    else:
        df_mid = massive_df.tail(leftovers).midRate.resample('15Min').ohlc()
        df_mid['vol'] = massive_df.tail(leftovers).bidAskVol.resample('15Min').mean()
        df_mid_ta = pad_history()
    massive_df.update(df_mid_ta)
    massive_df = massive_df.ffill().astype('float32')
    return massive_df, resampled_df

In [11]:
massive_df, resampled_df = pd.DataFrame(), pd.DataFrame()

In [12]:
for iteration in range(30):
    base_row = get_next_data_as_numpy_array(len(massive_df))
    row = compute_cross_sectional(base_row)
    massive_df = append_to_df(massive_df, row)
    massive_df, resampled_df = add_resample_features(massive_df, resampled_df)
    massive_df = add_time_features(massive_df)
    massive_df = add_manual_time_features(massive_df)

C:\Users\Tym\Miniconda3\envs\working\lib\site-packages\ta\trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
C:\Users\Tym\Miniconda3\envs\working\lib\site-packages\ta\trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
C:\Users\Tym\Miniconda3\envs\working\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [13]:
massive_df

,OIR,VOI,askRate0,askRate1,askRate10,askRate11,askRate12,askRate13,askRate14,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askRate8,askRate9,askSize0,askSize1,askSize10,askSize11,askSize12,askSize13,askSize14,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,bidAskRatio1,bidAskRatio10,bidAskRatio11,bidAskRatio12,bidAskRatio13,bidAskRatio14,bidAskRatio2,bidAskRatio3,bidAskRatio4,bidAskRatio5,bidAskRatio6,bidAskRatio7,bidAskRatio8,bidAskRatio9,bidAskVol,bidRate0,bidRate1,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidSize0,bidSize1,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,daskRate1,daskRate10,daskRate100,daskRate2,daskRate20,daskRate200,daskRate3,daskRate30,daskRate300,daskRate4,daskRate40,daskRate400,daskRate5,daskRate50,daskRate500,daskRate6,daskRate60,daskRate600,daskRate7,daskRate70,daskRate700,daskRate8,daskRate80,daskRate800,daskRate9,daskRate90,daskRate900,dbidRate1,dbidRate10,dbidRate100,dbidRate2,dbidRate20,dbidRate200,dbidRate3,dbidRate30,dbidRate300,dbidRate4,dbidRate40,dbidRate400,dbidRate5,dbidRate50,dbidRate500,dbidRate6,dbidRate60,dbidRate600,dbidRate7,dbidRate70,dbidRate700,dbidRate8,dbidRate80,dbidRate800,dbidRate9,dbidRate90,dbidRate900,deltaVAsk,deltaVBid,diff_vwaBidAskDMid,midRate,spread,totalAskVol1,totalAskVol10,totalAskVol11,totalAskVol12,totalAskVol13,totalAskVol14,totalAskVol2,totalAskVol3,totalAskVol4,totalAskVol5,totalAskVol6,totalAskVol7,totalAskVol8,totalAskVol9,totalAvailVol,totalBidVol1,totalBidVol10,totalBidVol11,totalBidVol12,totalBidVol13,totalBidVol14,totalBidVol2,totalBidVol3,totalBidVol4,totalBidVol5,totalBidVol6,totalBidVol7,totalBidVol8,totalBidVol9,vwaAsk,vwaAskDMid,vwaBid,vwaBidDMid
1970-01-01 00:00:00,0.75,6.0,1619.5,1620.0,0.0,0.0,0.0,0.0,0.0,1621.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545455,4.514286,4.800000,4.828571,5.114286,5.485714,0.514286,0.800000,1.371429,1.457143,2.028571,2.800000,3.114286,3.514286,8.0,1615.0,1614.0,1603.0,1602.0,1601.5,1601.0,1600.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,7.0,10.0,35.0,10.0,1.0,10.0,13.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,-7.695759,1617.25,4.5,11.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,227.0,17.0,158.0,168.0,169.0,179.0,192.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,1620.671387,3.421429,1606.132812,11.117188
1970-01-01 00:01:00,0.75,0.0,1619.5,1620.0,0.0,0.0,0.0,0.0,0.0,1621.0,1621.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,24.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545455,3.950000,4.200000,4.225000,4.475000,4.800000,0.514286,0.700000,1.200000,1.275000,1.775000,2.450000,2.725000,3.075000,8.0,1615.0,1614.0,1603.0,1602.0,1601.5,1601.0,1600.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,7.0,10.0,35.0,10.0,1.0,10.0,13.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.592187,1617.25,4.5,11.0,40.0,40.0,40.0,40.0,40.0,35.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,232.0,17.0,158.0,168.0,169.0,179.0,192.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,1620.775024,3.525000,1606.132812,11.117188
1970-01-01 00:02:00,0.75,0.0,1619.5,1620.0,0.0,0.0,0.0,0.0,0.0,1621.0,1621.5,1622.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,24.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,1.545455,3.761905,4.000000,4.023809,4.261905,4.571429,0.514286,0.700000,1.142857,1.214286,1.690476,2.333333,2.595238,2.928571,8.0,1615.0,1614.0,1603.0,1602.0,1601.5,1601.0,1600.